# Student Loan Risk with Deep Learning

In [1]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [2]:
# Read the csv into a Pandas DataFrame
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m18/lms/datasets/student-loans.csv"
loans_df = pd.read_csv(file_path)

# Review the DataFrame
loans_df.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


In [3]:
# Review the data types associated with the columns
loans_df.dtypes

payment_history           float64
location_parameter        float64
stem_degree_score         float64
gpa_ranking               float64
alumni_success            float64
study_major_code          float64
time_to_completion        float64
finance_workshop_score    float64
cohort_ranking            float64
total_loan_score          float64
financial_aid_score       float64
credit_ranking              int64
dtype: object

In [4]:
# Check the credit_ranking value counts
loans_df["credit_ranking"].value_counts()

credit_ranking
1    855
0    744
Name: count, dtype: int64

### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [5]:
# Define the target set y using the credit_ranking column
y = loans_df['credit_ranking']

# Display a sample of y
y[0:10]

0    0
1    0
2    0
3    1
4    0
5    0
6    0
7    1
8    1
9    0
Name: credit_ranking, dtype: int64

In [6]:
# Define features set X by selecting all columns but credit_ranking
X = loans_df.drop(columns='credit_ranking')

# Review the features DataFrame
X.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4


### Step 3: Split the features and target sets into training and testing datasets.


In [7]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [8]:
# Create a StandardScaler instance
X_scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_scaler.fit(X_train)

# Fit the scaler to the features training dataset
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [9]:
# Define the the number of inputs (features) to the model
input_nodes = len(X.columns)


In [10]:
# Define the number of hidden nodes for the first hidden layer
hn1 = 8

# Define the number of hidden nodes for the second hidden layer
hn2 = 6

# Define the number of neurons in the output layer
output_neurons = 1

In [11]:
# Create the Sequential model instance
nn_model = tf.keras.models.Sequential()

# Add the first hidden layer
nn_model.add(Dense(units=hn1, activation='relu', input_dim=input_nodes))

# Add the second hidden layer
nn_model.add(Dense(units=hn2, activation='relu'))

# Add the output layer to the model specifying the number of output neurons and activation function
nn_model.add(Dense(units=output_neurons, activation='sigmoid'))

In [12]:
# Display the Sequential model summary
nn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 96        
                                                                 
 dense_1 (Dense)             (None, 6)                 54        
                                                                 
 dense_2 (Dense)             (None, 1)                 7         
                                                                 
Total params: 157 (628.00 Byte)
Trainable params: 157 (628.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [13]:
# Compile the Sequential model
nn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [14]:
# Fit the model using 50 epochs and the training data
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
38/38 [==============================] - 0s 662us/step - loss: 0.7393 - accuracy: 0.5346
Epoch 2/50
38/38 [==============================] - 0s 554us/step - loss: 0.6645 - accuracy: 0.6130
Epoch 3/50
38/38 [==============================] - 0s 554us/step - loss: 0.6205 - accuracy: 0.6606
Epoch 4/50
38/38 [==============================] - 0s 514us/step - loss: 0.5892 - accuracy: 0.7023
Epoch 5/50
38/38 [==============================] - 0s 527us/step - loss: 0.5679 - accuracy: 0.7248
Epoch 6/50
38/38 [==============================] - 0s 548us/step - loss: 0.5550 - accuracy: 0.7281
Epoch 7/50
38/38 [==============================] - 0s 527us/step - loss: 0.5470 - accuracy: 0.7306
Epoch 8/50
38/38 [==============================] - 0s 716us/step - loss: 0.5413 - accuracy: 0.7331
Epoch 9/50
38/38 [==============================] - 0s 554us/step - loss: 0.5377 - accuracy: 0.7364
Epoch 10/50
38/38 [==============================] - 0s 554us/step - loss: 0.5353 - accuracy: 0.7323

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [15]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

13/13 - 0s - loss: 0.5094 - accuracy: 0.7475 - 70ms/epoch - 5ms/step
Loss: 0.5093716979026794, Accuracy: 0.7475000023841858


### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [16]:
# Set the model's file path
path = 'student_loans.keras'

# Export your model to a keras file
nn_model.save(path)

print(f"Model saved to {path}")


Model saved to student_loans.keras


---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [17]:
# Set the model's file path
path = 'student_loans.keras'

# Load the model to a new object
from tensorflow.keras.models import load_model
loaded_model = load_model(path)

# Verify the loaded model
loaded_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 96        
                                                                 
 dense_1 (Dense)             (None, 6)                 54        
                                                                 
 dense_2 (Dense)             (None, 1)                 7         
                                                                 
Total params: 157 (628.00 Byte)
Trainable params: 157 (628.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [19]:
# Make predictions with the test data
predictions = loaded_model.predict(X_test_scaled,verbose=2)

# Display a sample of the predictions
predictions

13/13 - 0s - 28ms/epoch - 2ms/step


array([[0.27490515],
       [0.42457902],
       [0.8466233 ],
       [0.760412  ],
       [0.95671254],
       [0.90906715],
       [0.9317096 ],
       [0.0610268 ],
       [0.25747353],
       [0.45189115],
       [0.89156705],
       [0.16905463],
       [0.6315119 ],
       [0.9302708 ],
       [0.5836133 ],
       [0.31627592],
       [0.8648264 ],
       [0.39447883],
       [0.6221051 ],
       [0.40806746],
       [0.54582757],
       [0.9046131 ],
       [0.21670252],
       [0.92029816],
       [0.39013112],
       [0.9124796 ],
       [0.860121  ],
       [0.64256185],
       [0.3016343 ],
       [0.27748674],
       [0.446377  ],
       [0.9700009 ],
       [0.21871965],
       [0.9508285 ],
       [0.20413901],
       [0.6484429 ],
       [0.1755234 ],
       [0.30398837],
       [0.92568314],
       [0.15902139],
       [0.9106524 ],
       [0.08619472],
       [0.15507029],
       [0.92336327],
       [0.10849506],
       [0.6474578 ],
       [0.2563194 ],
       [0.438

In [20]:
# Save the predictions to a DataFrame and round the predictions to binary results
predictions_df = pd.DataFrame((predictions > 0.5).astype("int32"), columns=["Predicted"])

# Display a sample of the predictions
predictions_df.sample(10)

,Predicted
157,1
273,1
45,1
331,0
352,1
296,0
351,1
382,1
181,1
254,1


### Step 4: Display a classification report with the y test data and predictions

In [21]:
# Print the classification report with the y test data and predictions
print(classification_report(y_test, predictions_df["Predicted"]))

              precision    recall  f1-score   support

           0       0.72      0.76      0.74       188
           1       0.78      0.74      0.76       212

    accuracy                           0.75       400
   macro avg       0.75      0.75      0.75       400
weighted avg       0.75      0.75      0.75       400



---
## Discuss creating a recommendation system for student loans


Briefly answer the following questions:
 1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.

	  > To build a recommendation system for student loan options, the
	  > following data would be relevant:
	  > 
	  > Student Demographics: Age, gender, location, and educational
	  > background. These factors can influence loan options and terms
	  > available to students.
	  > 
	  > Academic Information: Current level of study (undergraduate,
	  > graduate), major, GPA, and university attended. Lenders may offer
	  > different terms based on academic performance and institution
	  > reputation.
	  > 
	  > Financial Information: Income, existing debts, credit score, and
	  > financial aid status. These are critical for determining loan
	  > eligibility and interest rates.
	  > 
	  > Loan Preferences: Preferred loan amount, repayment period, and
	  > interest rate type (fixed or variable). This helps tailor
	  > recommendations to student preferences.
	  > 
	  > Historical Loan Data: Previous loans taken, repayment history, and
	  > loan default rates. Past behavior can predict future loan performance
	  > and suitability.
	  > 
	  > This data is relevant because it provides a comprehensive view of the
	  > student's financial situation, academic standing, and loan
	  > preferences, which are crucial for making accurate and personalized
	  > loan recommendations.

  

2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.

	> The model would primarily use content-based filtering. This method
	> focuses on the attributes of the items (in this case, loan options)
	> and the user's profile to make recommendations.
	> 
	> Justification: The selected data includes detailed attributes of both
	> the students (users) and the loans (items), such as financial
	> information, academic performance, and loan preferences. Content-based
	> filtering leverages these attributes to match students with the most
	> suitable loan options based on their individual profiles.
	> 
	> While collaborative filtering could also be used if we have access to
	> user interaction data (e.g., loans previously taken by similar
	> students), content-based filtering is more suitable given the
	> importance of matching specific loan terms and conditions to
	> individual student profiles.

3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.

	>**Data Privacy and Security:**
	 > *Concern:* Student loan data involves sensitive financial and personal
	> information. Ensuring data privacy and security is paramount to
	> protect students' identities and financial information from breaches
	> and misuse.
	> 
	> *Impact:* Breaches can lead to identity theft and financial fraud,
	> eroding trust in the recommendation system and potentially leading to
	> legal consequences.
	> 
	> **Bias and Fairness:**
	> *Concern:* The recommendation system must ensure fairness and avoid
	> biases that could disadvantage certain groups of students based on
	> their demographics, academic background, or financial status.
	> 
	> *Impact:* Biases can lead to discriminatory recommendations, where some
	> students may consistently receive less favorable loan options. This
	> can exacerbate existing inequalities and reduce the system's overall
	> credibility and effectiveness.
	> 
	> Addressing these challenges involves implementing robust data
	> protection measures and developing algorithms that are transparent and
	> fair, ensuring equitable access to loan opportunities for all
	> students.
